# Loading and modifying layers and layer collections

In [1]:
from datetime import datetime

import geoengine as ge

In [2]:
ge.initialize("http://localhost:3030/api", ("admin@localhost", "adminadmin"))

## Retrieve layer collections

In [3]:
root_collection = ge.layer_collection()
root_collection

Layer Collection
name: Layer Providers
description: All available Geo Engine layer providers
id: f2424474-ef24-4c18-ab84-68592e12ce48
provider id: 1c3b8042-300b-485c-95b5-0147d9dc068d
items: Layer Collection
name: Data Catalog
description: Catalog of data and workflows
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Personal Data Catalog
description: Catalog of your personal data and workflows.
id: root
provider id: cbb21ee3-d15d-45c5-a175-66964adf4e85

       Layer Collection
name: Copernicus Dataspace
description: 
id: datasets
provider id: 2e44e5ab-0f93-467b-8599-9b5b6339d5cd

       Layer Collection
name: Element 84 AWS STAC
description: Sentinel-2 L2A COGs hosted on AWS by Element 84
id: SentinelS2L2ACogs
provider id: 5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5

       Layer Collection
name: EBV Portal
description: Access to EBV Portal data
id: classes
provider id: 77d0bf11-986e-43f5-b11d-898321f1854c

       Layer Collection
name: GBIF
description: Access to GBIF occurrence data
id: select
provider id: 1c01dbb9-e3ab-f9a2-06f5-228ba4b6bf7a

       Layer Collection
name: Open Weather
description: Access to Open Weather data
id: collections
provider id: 0668d980-6c82-47c4-b1d9-1096f6770666

In [4]:
root_of_layerdb = root_collection.items[0].load()
root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [ ]:
test_collection = root_of_layerdb.items[0].load()
test_collection

Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: Natural Earth II – RGB
description: A raster with three bands for RGB visualization
id: 83866f7b-dcee-47b8-9242-e5636ceaf402
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: Ports in Germany
description: Natural Earth Ports point filtered with Germany polygon
id: b75db46e-2b9a-4a86-b33f-bc06a73cd711
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: Stacked Raster
description: A raster with two bands for testing
id: c078db52-2dc6-4838-ad75-340cefeab476
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [6]:
layer = test_collection.get_items_by_name(name="Ports in Germany")[0].load()
layer

Layer
name: Ports in Germany
description: Natural Earth Ports point filtered with Germany polygon
id: b75db46e-2b9a-4a86-b33f-bc06a73cd711
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
workflow: {'operator': {'params': {}, 'sources': {'points': {'type': 'OgrSource', 'params': {'data': 'ne_10m_ports', 'attributeProjection': None, 'attributeFilters': None}}, 'polygons': {'type': 'OgrSource', 'params': {'data': 'germany_outline', 'attributeProjection': None, 'attributeFilters': None}}}, 'type': 'PointInPolygonFilter'}, 'type': 'Vector'}
symbology: None
properties: []
metadata: {}

In [7]:
workflow = layer.as_workflow()

time = datetime.strptime("2014-04-01T12:00:00.000Z", "%Y-%m-%dT%H:%M:%S.%f%z")

data = workflow.get_dataframe(
    ge.QueryRectangle(
        ge.BoundingBox2D(-111.533203125, -4.482421875, 114.345703125, 73.388671875),
        ge.TimeInterval(time, time),
        ge.SpatialResolution(0.1, 0.1),
    )
)

data

,geometry,scalerank,name,natlscale,featurecla,website,start,end
0,POINT (8.70724 53.86472),8,Cuxhaven,5.0,Port,www.cuxport.de,NaT,NaT
1,POINT (9.83581 54.47361),8,Eckernforde,5.0,Port,www.stadtwerke-eckernfoerde.de,NaT,NaT
2,POINT (8.48952 53.4875),8,Nordenham,5.0,Port,None,NaT,NaT
3,POINT (7.36896 53.09639),8,Papenburg,5.0,Port,None,NaT,NaT
4,POINT (11.43916 53.90639),8,Wismar,5.0,Port,www.hafen-wismar.de,NaT,NaT
5,POINT (10.86826 53.95222),7,Travemunde,10.0,Port,None,NaT,NaT
6,POINT (7.19287 53.34611),6,Emden,20.0,Port,www.seaport-emden.de,NaT,NaT
7,POINT (9.43687 54.80333),6,Flensburg,20.0,Port,www.flensburg-tourismus.de,NaT,NaT
8,POINT (8.12214 53.53),6,Wilhelmshaven,20.0,Port,www.wilhelmshaven-port.de,NaT,NaT
9,POINT (8.75106 53.0975),5,Bremen,30.0,Port,www.bremen-ports.de,NaT,NaT


## Add collections and layers

In [8]:
root_of_layerdb.add_collection("my test collection", "test description")
root_of_layerdb = root_of_layerdb.reload()
root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: my test collection
description: test description
id: 26aed0be-5423-468d-8961-65c0c58d717a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [9]:
test_collection = root_of_layerdb.get_items_by_name("my test collection")[0].load()
test_collection

Layer Collection
name: my test collection
description: test description
id: 26aed0be-5423-468d-8961-65c0c58d717a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [10]:
test_collection.add_collection("sub collection", "another description")

test_collection.add_layer(
    name="ports clone",
    description="test description",
    workflow={
        "type": "Vector",
        "operator": {
            "type": "PointInPolygonFilter",
            "params": {},
            "sources": {
                "points": {
                    "type": "OgrSource",
                    "params": {"data": "ne_10m_ports", "attributeProjection": None, "attributeFilters": None},
                },
                "polygons": {
                    "type": "OgrSource",
                    "params": {"data": "germany_outline", "attributeProjection": None, "attributeFilters": None},
                },
            },
        },
    },
    symbology=None,
)

test_collection = test_collection.reload()
test_collection

Layer Collection
name: my test collection
description: test description
id: 26aed0be-5423-468d-8961-65c0c58d717a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub collection
description: another description
id: 5da6c563-5583-4d53-bda9-56943ff5d6e0
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: ff72b8e4-d951-467a-829b-04bbad52fcf6
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [11]:
sub_collection = test_collection.items[0].load()
sub_collection.add_existing_layer(test_collection.items[1])

sub_collection.add_collection("sub sub collection", "yet another description")

sub_collection = sub_collection.reload()
sub_collection

Layer Collection
name: sub collection
description: another description
id: 5da6c563-5583-4d53-bda9-56943ff5d6e0
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub sub collection
description: yet another description
id: 74a2698d-f254-4144-aee2-fc42089c518b
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: ff72b8e4-d951-467a-829b-04bbad52fcf6
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [12]:
test_collection.add_existing_collection(sub_collection.items[0])

test_collection = test_collection.reload()
test_collection

Layer Collection
name: my test collection
description: test description
id: 26aed0be-5423-468d-8961-65c0c58d717a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub collection
description: another description
id: 5da6c563-5583-4d53-bda9-56943ff5d6e0
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: sub sub collection
description: yet another description
id: 74a2698d-f254-4144-aee2-fc42089c518b
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: ff72b8e4-d951-467a-829b-04bbad52fcf6
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Delete previously added collections and layers

In [13]:
test_collection

Layer Collection
name: my test collection
description: test description
id: 26aed0be-5423-468d-8961-65c0c58d717a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub collection
description: another description
id: 5da6c563-5583-4d53-bda9-56943ff5d6e0
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: sub sub collection
description: yet another description
id: 74a2698d-f254-4144-aee2-fc42089c518b
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: ff72b8e4-d951-467a-829b-04bbad52fcf6
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [14]:
test_collection.remove_item(0)
test_collection

Layer Collection
name: my test collection
description: test description
id: 26aed0be-5423-468d-8961-65c0c58d717a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: sub sub collection
description: yet another description
id: 74a2698d-f254-4144-aee2-fc42089c518b
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: ff72b8e4-d951-467a-829b-04bbad52fcf6
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [15]:
test_collection.remove_item(0)
test_collection

Layer Collection
name: my test collection
description: test description
id: 26aed0be-5423-468d-8961-65c0c58d717a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer
name: ports clone
description: test description
id: ff72b8e4-d951-467a-829b-04bbad52fcf6
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [16]:
test_collection.remove_item(0)
test_collection

Layer Collection
name: my test collection
description: test description
id: 26aed0be-5423-468d-8961-65c0c58d717a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [17]:
root_of_layerdb.reload()

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: my test collection
description: test description
id: 26aed0be-5423-468d-8961-65c0c58d717a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

In [18]:
test_collection.remove()

In [19]:
root_of_layerdb = root_of_layerdb.reload()
root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Search for layer in layer collection

In [20]:
search_result = root_of_layerdb.search("ports")

print("Found…\n")
print(*search_result)

search_result[0].load()

Found…

Layer
name: Ports in Germany
description: Natural Earth Ports point filtered with Germany polygon
id: b75db46e-2b9a-4a86-b33f-bc06a73cd711
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74



Layer
name: Ports in Germany
description: Natural Earth Ports point filtered with Germany polygon
id: b75db46e-2b9a-4a86-b33f-bc06a73cd711
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
workflow: {'operator': {'params': {}, 'sources': {'points': {'type': 'OgrSource', 'params': {'data': 'ne_10m_ports', 'attributeProjection': None, 'attributeFilters': None}}, 'polygons': {'type': 'OgrSource', 'params': {'data': 'germany_outline', 'attributeProjection': None, 'attributeFilters': None}}}, 'type': 'PointInPolygonFilter'}, 'type': 'Vector'}
symbology: None
properties: []
metadata: {}

In [21]:
root_of_layerdb.search("ports")[0].load()

Layer
name: Ports in Germany
description: Natural Earth Ports point filtered with Germany polygon
id: b75db46e-2b9a-4a86-b33f-bc06a73cd711
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
workflow: {'operator': {'params': {}, 'sources': {'points': {'type': 'OgrSource', 'params': {'data': 'ne_10m_ports', 'attributeProjection': None, 'attributeFilters': None}}, 'polygons': {'type': 'OgrSource', 'params': {'data': 'germany_outline', 'attributeProjection': None, 'attributeFilters': None}}}, 'type': 'PointInPolygonFilter'}, 'type': 'Vector'}
symbology: None
properties: []
metadata: {}